In [1]:
import random
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

/opt/anaconda3/envs/voice-chat-python3-11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def voice(filename, duration=5, fs=44100):
    print('Recording...')
    myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    print('Recording done')
    wav.write(filename, fs, myrecording)

In [6]:
def voice_to_text(user_audio_file):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)
    
    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )

    result = pipe(user_audio_file)
    print("text from voice: ")
    print(result["text"])

In [7]:
unique_id = ''.join(random.choices('0123456789abcdef', k=6))
print(unique_id)
user_audio_file = f"recorded-files/user_audio{unique_id}.wav"
# save the recorded sound in the location
voice(user_audio_file)
voice_to_text(user_audio_file)

0960f5
Recording...
Recording done


/opt/anaconda3/envs/voice-chat-python3-11/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behav

text from voice: 
 Hello, Joey Gao.
